In [1]:
from langchain.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain import PromptTemplate, LLMChain
from langchain.document_loaders import PyPDFLoader
from openai import AzureOpenAI
import os

In [2]:
def get_document_information(question: str) -> str:
    """Use this tool to get information about the QRISK3 method for cardiovascular risk prediction.
     
    The input to the function is a question, such as "Why is corticosteroids included in the QRISK3 prediction model?"
    The function returns a string containing the reasons explaining the answer.
    """
    loader = PyPDFLoader("rag_docs/Cadillac_CTS.pdf")
    pages = loader.load_and_split()

    embeddings = AzureOpenAIEmbeddings(
        azure_deployment="MIMIC_text_embedding",
        openai_api_version="2024-02-15-preview",
    )

    faiss_index = FAISS.from_documents(pages, embeddings)
    docs = faiss_index.similarity_search(question)
    # Get the best matched page
    full_text = docs[0].page_content

    # Summarize the information
    prompt = f"""Using the following information below, you are asked to provide answers to the question: {question}?

    {full_text}

    Answer:"""
    
    client = client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
    api_version="2024-02-15-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    )
    
    response = client.chat.completions.create(
        model="MIMIC",
        messages=[
            {"role": "system", "content": f"{prompt}"},
            {"role": "user", "content": f"{full_text}"}
        ]
    )

    answer = response.choices[0].message.content

    return answer

In [3]:
get_document_information("Is Cadillac CTS still in production?")

'No, the Cadillac CTS is no longer in production as of 2019. It was replaced by the Cadillac CT5.'

In [4]:
get_document_information("Is Audi a good car brand?")

"Based on the provided information, there is no direct mention or evidence to support a statement about the quality of Audi as a car brand. The information consists of articles and mentions related to the Cadillac CTS, including its recognition and awards from Motor Trend magazine. Therefore, it is not possible to provide a definitive answer to the question about Audi's reputation as a good car brand based on this information."